In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
from datetime import datetime 
import numpy as np
import os, random, math, time
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
def read_file(filename):
    df = pd.read_csv(f'{filename}', encoding="utf-8") #"cp949"
    if sum(df.duplicated(df.columns)) != 0:
        print(f"...There are {sum(df.duplicated(df.columns))} duplicates\n...Remove them")
        df = df.drop_duplicates()
    else:
        print("...There are no duplicates")
    df.name = filename
    print(f"shape: {df.shape}")
    return df

def read_file2(filename):
    df = pd.read_excel(f'{filename}', header=None)
    if sum(df.duplicated(df.columns)) != 0:
        print(f"...There are {sum(df.duplicated(df.columns))} duplicates\n...Remove them")
        df = df.drop_duplicates()
    else:
        print("...There are no duplicates")
    df.name = filename
    print(f"shape: {df.shape}")
    return df

def check_NA(df):
    print(f"NA check")
    NA_res = df.isna().sum()
    if NA_res.sum() != 0:
        for idx, i in enumerate(NA_res):
            if i != 0:
                print(df.columns[idx],i,"->",round(i/df.shape[0]*100,3),"%")
    else:
        print("...There's no NA")

In [3]:
filepath = "./data"

In [4]:
survey = read_file2(os.path.join(filepath,'설문데이터.xlsx'))

...There are no duplicates
shape: (603, 138)


In [5]:
df = deepcopy(survey)
print(df.shape)
df_info = df.iloc[0]
df.columns = df.iloc[1]
df = df.drop(index = [0, 1])
df = df.reset_index(drop=True)
df.head(3)

(603, 138)


1,id,id_guro,GP,sub_gp,SERIAL,born,age,sex,type1,type2,type3,pa1,pa2,pa3,pa4,pa5,pa6,pa7,pa8,pa9,...,j1,j2,j3,j4,reli,time1,time2,time3,time4,time5,dq7_1,dq7_2,dq7_3,dq7_4,dq7_5,dq7_6,dq7_7,DISE1,DISE2,DISE3
0,NaN,"125,321.0000",광양,NaN,02N190200348,"1,935.0000",85.0000,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,보통,그렇지 않다,그렇다,보통,3.0000,06:00:00,08:00:00,12:00:00,18:00:00,22:00:00,NaN,아침 식후,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"125,359.0000",광양,NaN,02N190200262,"1,941.0000",79.0000,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,그렇다,그렇다,그렇다,그렇다,3.0000,05:00:00,07:00:00,12:00:00,18:00:00,21:00:00,NaN,아침 식후,NaN,NaN,NaN,저녁 식후,NaN,NaN,NaN,NaN
2,NaN,209.0000,구로,구로어르신돌봄통합센터,NaN,"1,949.0000",70.0000,여,기초생활수급,독거,노인돌봄기본서비스,예,아니오,아니오,아니오,아니오,아니오,아니오,예,예,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,아침 식후,NaN,NaN,NaN,저녁 식후,NaN,NaN,NaN,NaN


# id와 id guro 둘다 특정 안되는 경우는 drop

In [6]:
set(df.id.unique())&set(df.id_guro.unique())

set()

- 겹치는 아이디는 없고

In [7]:
print(df.shape, df.id.nunique())

(601, 138) 179


In [8]:
df.loc[df.id =='126758, 126401', "id"]= "126758"
df = df.append(df.loc[190], ignore_index=True)
df.iloc[601, 0]= "126401"

In [9]:
df["id"]= df["id"].fillna(0)
df["id_guro"]= df["id_guro"].fillna(0)

df["id"]= df.id.astype("float")
df["id_guro"]= df.id_guro.astype("float")
df["id"]= df.id.apply(round)
df["id_guro"]= df.id_guro.apply(round)

df["id"]= df.id.astype("str")
df["id_guro"]= df.id_guro.astype("str")

In [10]:
def filter_na(x): 
    if len(x)<6: return "제거"
    else: return x

In [11]:
df["id"] = df['id'].apply(filter_na)
df["id_guro"] = df['id_guro'].apply(filter_na)
df = df[(df.id!="제거")|(df.id_guro!="제거")].reset_index(drop=True)

In [12]:
df[(df.id!="제거")&(df.id_guro!="제거")]

1,id,id_guro,GP,sub_gp,SERIAL,born,age,sex,type1,type2,type3,pa1,pa2,pa3,pa4,pa5,pa6,pa7,pa8,pa9,...,j1,j2,j3,j4,reli,time1,time2,time3,time4,time5,dq7_1,dq7_2,dq7_3,dq7_4,dq7_5,dq7_6,dq7_7,DISE1,DISE2,DISE3


In [13]:
df["id"]= df.id+"."+df.id_guro

In [14]:
def filter_na2(x): 
    a = x.split(".")[0]
    b = x.split(".")[1]
    if len(a)<len(b): return b
    else: return a

In [15]:
df["id"] = df['id'].apply(filter_na2)
df = df.drop(["id_guro"], axis=1)

In [16]:
print(df.id.nunique(), df.shape)

459 (459, 137)


In [17]:
df = df.rename(columns = {"id": "doll_id"})

In [18]:
df["doll_id"] = df.doll_id.astype("int")

In [19]:
df.head(3)

1,doll_id,GP,sub_gp,SERIAL,born,age,sex,type1,type2,type3,pa1,pa2,pa3,pa4,pa5,pa6,pa7,pa8,pa9,pa10,...,j1,j2,j3,j4,reli,time1,time2,time3,time4,time5,dq7_1,dq7_2,dq7_3,dq7_4,dq7_5,dq7_6,dq7_7,DISE1,DISE2,DISE3
0,125321,광양,NaN,02N190200348,"1,935.0000",85.0000,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,보통,그렇지 않다,그렇다,보통,3.0000,06:00:00,08:00:00,12:00:00,18:00:00,22:00:00,NaN,아침 식후,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,125359,광양,NaN,02N190200262,"1,941.0000",79.0000,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,그렇다,그렇다,그렇다,그렇다,3.0000,05:00:00,07:00:00,12:00:00,18:00:00,21:00:00,NaN,아침 식후,NaN,NaN,NaN,저녁 식후,NaN,NaN,NaN,NaN
2,126917,구로,구로어르신돌봄통합센터,NaN,"1,936.0000",83.0000,여,차상위,독거,노인돌봄기본서비스,예,예,아니오,아니오,아니오,아니오,아니오,아니오,예,아니오,...,그렇다,그렇다,그렇다,그렇다,NaN,NaN,NaN,NaN,NaN,NaN,NaN,아침 식후,NaN,NaN,NaN,저녁 식후,NaN,NaN,NaN,NaN


# column 종류 나누고 변수 유형에 맞게 변환

In [20]:
pd.set_option('display.max_rows', None)

In [21]:
df_info = pd.DataFrame(df_info)
df_info = df_info[1:]
df_info["col"]= df.columns
df_info.columns = ["info", "col"]

In [22]:
df_info[:10]

,info,col
1,NaN,doll_id
2,기관,GP
3,기관2,sub_gp
4,제품시리얼,SERIAL
5,생년,born
6,연령(조사시점 연령),age
7,성별,sex
8,수급여부,type1
9,세대구성,type2
10,제공받는 서비스,type3


In [23]:
# 기본 정보 - id, serial 제외
col_info = ['GP', 'sub_gp', 'born', 'age', 'sex', 'reli', 'type1', 'type2','type3', 
            'house', 'dq1', 'dq2', 'dq3', 'dq3_m', 'dq3_f', 'dq4', 'dq5', 'dq6']

# 기본 정보
col_pa = ['pa1', 'pa2', 'pa3', 'pa4', 'pa5', 'pa6', 'pa7', 'pa8', 'pa9','pa10']

# 복용 약과 복용 시간
col_med =  ['dq8_1', 'dq8_2', 'dq8_3', 'dq8_4', 'dq8_5', 'dq8_6', 'dq8_7'] # 복용약
col_med_t = ['dq7_1', 'dq7_2','dq7_3', 'dq7_4', 'dq7_5', 'dq7_6', 'dq7_7'] # 복용 시간 - 복용약 별로 시간이 있는 것은 아님
col_dis = ['DISE1', 'DISE2', 'DISE3'] # all nan

# 우울증 설문: 인형 사용 전후
col_psy_before = ['sgd1_P', 'sgd2', 'sgd3', 'sgd4', 'sgd5_P', 'sgd6', 'sgd7_P', 'sgd8',
                  'sgd9', 'sgd10', 'sgd11_P', 'sgd12', 'sgd13_P', 'sgd14', 'sgd15'] 
col_psy_after = ['P_sgd1_P','P_sgd2', 'P_sgd3', 'P_sgd4', 'P_sgd5_P', 'P_sgd6', 'P_sgd7_P',
                 'P_sgd8', 'P_sgd9', 'P_sgd10', 'P_sgd11_P', 'P_sgd12', 'P_sgd13_P','P_sgd14', 'P_sgd15']

# 시니어 생활관리 : 인형 사용 전후
col_life1_before = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11','A12'] # 점수가 낮으면 좋음
col_life1_after = ['p_a1', 'p_a2', 'p_a3', 'p_a4', 'p_a5', 'p_a6','p_a7', 'p_a8', 'p_a9', 'p_a10', 'p_a11', 'p_a12']

col_life2_before = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8'] #항목에 따라 3점 척도, 점수가 높으면 좋음
col_life2_after = ['p_d1', 'p_d2','p_d3', 'p_d4', 'p_d5', 'p_d6', 'P_D7', 'P_D8']

# 인형 사용 만족도 관련
col_doll = ['h1', 'i1', 'i2', 'i3','i4', 'i5', 'i6', 'i7', 'i8', 'i9', 'i10', 'j1', 'j2', 'j3', 'j4']

# 인형 사용 시간
col_time = ['time1', 'time2', 'time3', 'time4', 'time5']

In [24]:
df.columns

Index(['doll_id', 'GP', 'sub_gp', 'SERIAL', 'born', 'age', 'sex', 'type1',
       'type2', 'type3',
       ...
       'dq7_1', 'dq7_2', 'dq7_3', 'dq7_4', 'dq7_5', 'dq7_6', 'dq7_7', 'DISE1',
       'DISE2', 'DISE3'],
      dtype='object', name=1, length=137)

In [25]:
df = df[["doll_id", 
        'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', # 시니어 생활관리 척도 사용 전
        'p_d1', 'p_d2','p_d3', 'p_d4', 'p_d5', 'p_d6', 'P_D7', 'P_D8' # 시니어 생활관리 척도 사용 후 
        ]]

In [26]:
for i in col_life2_before: 
    print(df[i].value_counts())

매일 규칙적      190
때에 따라       118
일정 하지 않음     87
0.0           6
Name: D1, dtype: int64
매일 규칙적      323
때에 따라        63
일정 하지 않음      6
0.0           6
Name: D2, dtype: int64
매일 정확한 시간에 약을 복용하고 있음     313
가끔 약 먹는 것을 잊어 먹을 때가 있음     70
0.0                         6
거의 매번 약 먹는 것을 잊어 먹음         3
Name: D3, dtype: int64
매일 규칙적 세끼 식사하고 있음    234
가끔 식사시간 놓침           152
거의 매번 식사 시간 놓침         9
0.0                    6
Name: D4, dtype: int64
매일 규칙적      169
때에 따라       146
일정 하지 않음     80
0.0           6
Name: D5, dtype: int64
때에 따라       148
매일 규칙적      124
일정 하지 않음    123
0.0           6
Name: D6, dtype: int64
매일 규칙적      193
일정 하지 않음    105
때에 따라        97
Name: D7, dtype: int64
매우 긍정적    232
조금 있음      99
거의 없음      64
Name: D8, dtype: int64


In [27]:
for i in col_life2_after: 
    print(df[i].value_counts())

매일 규칙적      89
일정 하지 않음    52
때에 따라       29
Name: p_d1, dtype: int64
매일 규칙적      158
때에 따라        11
일정 하지 않음      1
Name: p_d2, dtype: int64
매일 정확한 시간에 약을 복용하고 있음     164
가끔 약 먹는 것을 잊어 먹을 때가 있음      6
Name: p_d3, dtype: int64
매일 규칙적 세끼 식사하고 있음    144
가끔 식사시간 놓침            22
거의 매번 식사 시간 놓침         4
Name: p_d4, dtype: int64
매일 규칙적      79
때에 따라       67
일정 하지 않음    24
Name: p_d5, dtype: int64
매일 규칙적      70
때에 따라       69
일정 하지 않음    31
Name: p_d6, dtype: int64
일정 하지 않음    85
매일 규칙적      58
때에 따라       27
Name: P_D7, dtype: int64
3.0000    96
1.0000    38
2.0000    36
Name: P_D8, dtype: int64


# 시니어 생활관리: 3, 2, 1 척도. => 점수로 전처리 후 점수/카테고리만 남기기
## 점수가 높은게 좋은 항목

In [28]:
for i in col_life2_before: 
    df[i] = df[i].replace("매일 규칙적", 3)
    df[i] = df[i].replace("매일 정확한 시간에 약을 복용하고 있음", 3)
    df[i] = df[i].replace("매일 규칙적 세끼 식사하고 있음", 3)
    df[i] = df[i].replace("매일 긍정적", 3)
    df[i] = df[i].replace("매우 긍정적", 3)
    df[i] = df[i].replace("매일 다른 사람들과 접촉하고 싶음", 3)
    
    df[i] = df[i].replace("때에 따라", 2)
    df[i] = df[i].replace("가끔 필요시", 2)
    df[i] = df[i].replace("조금 있음", 2)
    df[i] = df[i].replace("가끔 약 먹는 것을 잊을 때가 있음", 2)
    df[i] = df[i].replace("가끔 약 먹는 것을 잊어 먹을 때가 있음", 2)
    df[i] = df[i].replace("가끔 식사시간 놓침", 2)
    df[i] = df[i].replace("가끔 부정적인 생각이 들 때가 있음", 2)
    df[i] = df[i].replace("가끔 다른 사람들과 접촉하고 싶음", 2)
    
    df[i] = df[i].replace("일정 하지 않음", 1)
    df[i] = df[i].replace("거의 하지 않음", 1)
    df[i] = df[i].replace("거의 없음", 1)
    df[i] = df[i].replace("거의 매번 약 먹는 것을 잊음", 1)
    df[i] = df[i].replace("거의 매번 약 먹는 것을 잊어 먹음", 1)
    df[i] = df[i].replace("거의 매번 식사시간 놓침", 1)
    df[i] = df[i].replace("거의 매번 식사 시간 놓침", 1)
    df[i] = df[i].replace("긍정적인 생각을 거의 하지 않음", 1)
    df[i] = df[i].replace("다른 사람들과 접촉하고 싶지 않음", 1)

In [29]:
for i in col_life2_after: 
    df[i] = df[i].replace("매일 규칙적", 3)
    df[i] = df[i].replace("매일 정확한 시간에 약을 복용하고 있음", 3)
    df[i] = df[i].replace("매일 규칙적 세끼 식사하고 있음", 3)
    df[i] = df[i].replace("매일 긍정적", 3)
    df[i] = df[i].replace("매우 긍정적", 3)
    df[i] = df[i].replace("매일 다른 사람들과 접촉하고 싶음", 3)
    
    df[i] = df[i].replace("때에 따라", 2)
    df[i] = df[i].replace("가끔 필요시", 2)
    df[i] = df[i].replace("조금 있음", 2)
    df[i] = df[i].replace("가끔 약 먹는 것을 잊을 때가 있음", 2)
    df[i] = df[i].replace("가끔 약 먹는 것을 잊어 먹을 때가 있음", 2)
    df[i] = df[i].replace("가끔 식사시간 놓침", 2)
    df[i] = df[i].replace("가끔 부정적인 생각이 들 때가 있음", 2)
    df[i] = df[i].replace("가끔 다른 사람들과 접촉하고 싶음", 2)
    
    df[i] = df[i].replace("일정 하지 않음", 1)
    df[i] = df[i].replace("거의 하지 않음", 1)
    df[i] = df[i].replace("거의 없음", 1)
    df[i] = df[i].replace("거의 매번 약 먹는 것을 잊음", 1)
    df[i] = df[i].replace("거의 매번 약 먹는 것을 잊어 먹음", 1)
    df[i] = df[i].replace("거의 매번 식사시간 놓침", 1)
    df[i] = df[i].replace("거의 매번 식사 시간 놓침", 1)
    df[i] = df[i].replace("긍정적인 생각을 거의 하지 않음", 1)
    df[i] = df[i].replace("다른 사람들과 접촉하고 싶지 않음", 1)

In [30]:
for i in col_life2_after:
    print(i, df[i].value_counts())

p_d1 3.0000    89
1.0000    52
2.0000    29
Name: p_d1, dtype: int64
p_d2 3.0000    158
2.0000     11
1.0000      1
Name: p_d2, dtype: int64
p_d3 3.0000    164
2.0000      6
Name: p_d3, dtype: int64
p_d4 3.0000    144
2.0000     22
1.0000      4
Name: p_d4, dtype: int64
p_d5 3.0000    79
2.0000    67
1.0000    24
Name: p_d5, dtype: int64
p_d6 3.0000    70
2.0000    69
1.0000    31
Name: p_d6, dtype: int64
P_D7 1.0000    85
3.0000    58
2.0000    27
Name: P_D7, dtype: int64
P_D8 3.0000    96
1.0000    38
2.0000    36
Name: P_D8, dtype: int64


In [31]:
df = df.reset_index(drop=True)

In [32]:
dat = read_file(os.path.join(filepath,'surveydata_md.csv'))

...There are no duplicates
shape: (160, 87)


In [33]:
df = pd.merge(dat, df, on='doll_id', how='inner')

In [34]:
slife_list = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'p_d1', 'p_d2', 'p_d3', 'p_d4', 'p_d5', 'p_d6', 'P_D7', 'P_D8']

In [35]:
for i in slife_list: 
    df[i] = df[i].astype('int')

In [36]:
df

,doll_id,stroke,hand_hold_sum,knock,human_detection,gymnastics,brain_timer,drug_consume,emrg_cnt,yn,drug_consume_rt,battery,active_monitor,종교,기상,아침,점심,저녁,취침,is_edited,...,우울증_변화1,우울증_변화2,생활관리_변화1,생활관리_변화2,D1,D2,D3,D4,D5,D6,D7,D8,p_d1,p_d2,p_d3,p_d4,p_d5,p_d6,P_D7,P_D8
0,126401,"1,442.0000","1,908.0000","4,631.0000","50,060.0000",59.0000,59.0000,97.0000,0.0000,82,0.9175,92,24,불교,8,9,12,18,21,0,...,bad,bad,good,same,3,3,3,2,1,1,1,2,2,3,3,3,2,1,1,1
1,126497,65.0000,47.0000,209.0000,"9,095.0000",14.0000,11.0000,60.0000,3.0000,67,0.8083,100,24,무교,6,7,12,17,22,0,...,good,same,good,good,1,3,2,3,3,1,3,3,3,3,3,3,3,3,1,3
2,126498,1.0000,0.0000,2.0000,152.0000,0.0000,0.0000,4.0000,2.0000,12,1.0000,100,24,무교,6,7,12,18,20,0,...,good,same,bad,same,3,2,3,3,3,2,1,1,1,3,3,3,2,1,1,3
3,126500,341.0000,199.0000,441.0000,"23,421.0000",20.0000,31.0000,7.0000,0.0000,63,1.0000,91,24,불교,7,9,13,19,23,0,...,bad,bad,bad,same,2,3,3,2,3,3,3,2,3,3,3,3,3,2,1,2
4,126501,8.0000,27.0000,22.0000,"4,255.0000",6.0000,6.0000,21.0000,1.0000,85,1.0000,100,24,불교,4,8,12,18,21,0,...,good,good,bad,same,3,3,3,3,1,1,3,3,3,3,3,3,2,1,1,3
5,126502,276.0000,377.0000,"1,063.0000","30,576.0000",88.0000,97.0000,49.0000,4.0000,98,0.6020,100,24,기독교,6,10,14,19,23,0,...,good,good,good,good,3,3,3,3,1,1,1,1,3,3,3,1,2,3,1,3
6,126549,57.0000,113.0000,88.0000,"5,839.0000",3.0000,8.0000,0.0000,1.0000,35,0.0000,100,24,천주교,5,8,12,19,22,0,...,bad,same,bad,bad,2,3,2,3,3,3,3,3,3,3,3,2,1,3,2,3
7,126550,200.0000,436.0000,44.0000,"2,448.0000",19.0000,25.0000,29.0000,18.0000,78,0.5000,94,24,무교,6,7,9,18,22,0,...,good,same,good,good,1,3,3,2,1,1,1,2,3,3,3,3,3,2,1,3
8,126552,206.0000,214.0000,216.0000,"3,814.0000",12.0000,31.0000,80.0000,4.0000,85,0.8750,14,24,무교,6,10,12,21,22,0,...,good,same,good,same,3,3,3,2,1,1,1,2,1,3,3,2,1,2,1,3
9,126553,107.0000,93.0000,114.0000,"63,936.0000",5.0000,2.0000,85.0000,16.0000,82,0.8647,1,24,천주교,7,9,12,18,0,0,...,good,good,good,good,1,3,2,2,1,2,1,1,1,3,3,3,2,2,1,3


In [37]:
check_NA(df)

NA check
...There's no NA


In [38]:
df

,doll_id,stroke,hand_hold_sum,knock,human_detection,gymnastics,brain_timer,drug_consume,emrg_cnt,yn,drug_consume_rt,battery,active_monitor,종교,기상,아침,점심,저녁,취침,is_edited,...,우울증_변화1,우울증_변화2,생활관리_변화1,생활관리_변화2,D1,D2,D3,D4,D5,D6,D7,D8,p_d1,p_d2,p_d3,p_d4,p_d5,p_d6,P_D7,P_D8
0,126401,"1,442.0000","1,908.0000","4,631.0000","50,060.0000",59.0000,59.0000,97.0000,0.0000,82,0.9175,92,24,불교,8,9,12,18,21,0,...,bad,bad,good,same,3,3,3,2,1,1,1,2,2,3,3,3,2,1,1,1
1,126497,65.0000,47.0000,209.0000,"9,095.0000",14.0000,11.0000,60.0000,3.0000,67,0.8083,100,24,무교,6,7,12,17,22,0,...,good,same,good,good,1,3,2,3,3,1,3,3,3,3,3,3,3,3,1,3
2,126498,1.0000,0.0000,2.0000,152.0000,0.0000,0.0000,4.0000,2.0000,12,1.0000,100,24,무교,6,7,12,18,20,0,...,good,same,bad,same,3,2,3,3,3,2,1,1,1,3,3,3,2,1,1,3
3,126500,341.0000,199.0000,441.0000,"23,421.0000",20.0000,31.0000,7.0000,0.0000,63,1.0000,91,24,불교,7,9,13,19,23,0,...,bad,bad,bad,same,2,3,3,2,3,3,3,2,3,3,3,3,3,2,1,2
4,126501,8.0000,27.0000,22.0000,"4,255.0000",6.0000,6.0000,21.0000,1.0000,85,1.0000,100,24,불교,4,8,12,18,21,0,...,good,good,bad,same,3,3,3,3,1,1,3,3,3,3,3,3,2,1,1,3
5,126502,276.0000,377.0000,"1,063.0000","30,576.0000",88.0000,97.0000,49.0000,4.0000,98,0.6020,100,24,기독교,6,10,14,19,23,0,...,good,good,good,good,3,3,3,3,1,1,1,1,3,3,3,1,2,3,1,3
6,126549,57.0000,113.0000,88.0000,"5,839.0000",3.0000,8.0000,0.0000,1.0000,35,0.0000,100,24,천주교,5,8,12,19,22,0,...,bad,same,bad,bad,2,3,2,3,3,3,3,3,3,3,3,2,1,3,2,3
7,126550,200.0000,436.0000,44.0000,"2,448.0000",19.0000,25.0000,29.0000,18.0000,78,0.5000,94,24,무교,6,7,9,18,22,0,...,good,same,good,good,1,3,3,2,1,1,1,2,3,3,3,3,3,2,1,3
8,126552,206.0000,214.0000,216.0000,"3,814.0000",12.0000,31.0000,80.0000,4.0000,85,0.8750,14,24,무교,6,10,12,21,22,0,...,good,same,good,same,3,3,3,2,1,1,1,2,1,3,3,2,1,2,1,3
9,126553,107.0000,93.0000,114.0000,"63,936.0000",5.0000,2.0000,85.0000,16.0000,82,0.8647,1,24,천주교,7,9,12,18,0,0,...,good,good,good,good,1,3,2,2,1,2,1,1,1,3,3,3,2,2,1,3


In [39]:
df.to_csv('data/surveydata_md2.csv', index=False)